In [ ]:
import tensorflow as tf
import pandas as pd

import numpy as np

## Exploratory Data Analysis

## Data Pre-processing and Augmentation

## Plate position Detection Model

### Model Definition

#### Pre-trained model

#### Custom CNN

### Model Training

### Model Evaluation

## OCR for text recognition